# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import gmaps.datasets
import os

# Import API key
from config import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# import csv from part 1 and load to a dataframe
cities_data_df = pd.read_csv("../WeatherPy/output_data/cities.csv", encoding="utf-8")
cities_data_df

# Check data types
print(cities_data_df.dtypes)

Unnamed: 0      int64
City           object
Latitude      float64
Longitude     float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
Country        object
Date            int64
dtype: object


In [3]:
# Drop anything that may be blank
cities_data_df = cities_data_df.dropna()
cities_data_df.head()

,Unnamed: 0,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,palmer,42.16,-72.33,69.01,100,1,2.62,US,1595651054
1,1,lasa,46.62,10.70,52.00,93,75,4.70,IT,1595651054
2,2,jamestown,42.10,-79.24,66.99,93,40,3.51,US,1595651054
3,3,provideniya,64.38,-173.30,50.81,69,99,3.58,RU,1595651054
4,4,ushuaia,-54.80,-68.30,28.40,74,75,13.87,AR,1595650921


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)


In [5]:
# Store latitude and longitude into locations
locations = cities_data_df[["Latitude","Longitude"]]

# Store humidity
# humidity = cities_data_df[["Humidity"]].astype(float)
humidity = pd.to_numeric(cities_data_df['Humidity'], downcast='float')



In [26]:
# Create heatmap layer
coordinates=(0.0,0.0)

figure_layout = {
    'width': '600px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px'
}

fig = gmaps.figure(layout=figure_layout, center=coordinates, zoom_level=1, map_type='SATELLITE')

heatmap_layer = gmaps.heatmap_layer(locations, weights=humidity, point_radius=8)
fig.add_layer(heatmap_layer)

fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', padding='1px', width='600px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [40]:
# Create dataframe to narrow down to ideal weather conditions
ideal_weather = cities_data_df.loc[((cities_data_df['Max Temp'] > 72) & (cities_data_df['Max Temp'] < 80)) & \
                                  (cities_data_df['Wind Speed'] < 10) & (cities_data_df['Cloudiness'] == 0)]
ideal_weather

,Unnamed: 0,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
136,136,saskylakh,-24.87,113.63,78.80,22,0,9.17,AU,1595651073
181,181,madang,-20.10,146.27,75.18,43,0,4.07,AU,1595651079
266,266,zwedru,31.21,16.59,75.54,81,0,5.19,LY,1595651091
270,270,saleaula,43.75,-81.72,73.99,85,0,7.11,CA,1595651092
302,302,karimnagar,37.47,-4.44,79.00,40,0,2.59,ES,1595651096
378,378,puerto leguizamo,38.20,20.43,73.40,69,0,4.76,GR,1595651107
403,403,rungata,28.10,-16.68,73.99,82,0,4.70,ES,1595651111
445,445,singtam,38.94,-3.24,75.99,38,0,2.39,ES,1595651116
447,447,saint george,37.15,122.38,77.99,67,0,8.43,CN,1595651116
499,499,tasiilaq,36.64,33.44,79.50,29,0,7.54,TR,1595651124


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [45]:
# geocoordinates
target_coordinates = ideal_weather[["Latitude","Longitude"]]
target_radius = 5000
target_type = "lodging"

# set up parameters dictionary
params= {
    "location": target_coordinates,
    "radius": target_radius,
    "type": target_type,
    "key": g_key
}

# create base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

response = requests.get(base_url, params=params)


In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
